# 4colab

In [7]:
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

# %cd '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# !ls

# data_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# result_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/result'

In [8]:
data_dir = 'data'
result_dir = 'result'

# Import

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

import warnings
warnings.filterwarnings("ignore")

# Read data

In [10]:
af = 'vv'
dt = 'v'
pf = '__5__'

Xtrain = np.load(f'{data_dir}/{af}3.tr.X.{pf}.npy', allow_pickle=True)
y_train = np.load(f'{data_dir}/{af}3.tr.y.npy', allow_pickle=True)

Xtest = np.load(f'{data_dir}/{af}3.t.X.{pf}.npy', allow_pickle=True)
y_test = np.load(f'{data_dir}/{af}3.t.y.npy', allow_pickle=True)

# Xval = np.load(f'{data_dir}/{af}3.v.X.{pf}.npy', allow_pickle=True)
# y_val = np.load(f'{data_dir}/{af}3.v.y.npy', allow_pickle=True)

#? features names
fts_names = [line.strip() for line in open(f'{data_dir}/{af}3.fts_cols.{pf}.txt').readlines()]

In [11]:
Xtrain.shape

(1529165, 65)

In [12]:
# for i in range(0, len(fts_names)):
#     print(i, fts_names[i])

# Kmeans functions

In [37]:
from scipy.spatial.distance import euclidean
from scipy.stats import median_abs_deviation

def calculate_modifed_z_score(array, medians, mads):
    """
    This function is made to be used with `numpy.apply_along_axis`. Takes in a 2-dimensional 
    vector of distances and labels and returns their respective Modified Z-score acording to 
    their respective distribution.
    """
    distance, label = array
    median = medians[label]
    mad = mads[label]
    z_mod = (distance - median)/mad
    return z_mod

def get_distance_centroid(array, centroids):
    """
    This function is made to be used with `numpy.apply_along_axis`.Takes in an n-dimensional 
    vector of features (previously processed) and finds the distance to their respective 
    cluster centroid.
    """
    vector = array[:-1]
    label = array[-1]
    centroid = centroids[int(label)]
    dist = euclidean(vector, centroid)
    return dist

def characterization_k_clusters(kmeans_model, X, features_name):
    """
    """
    arr_labels = kmeans_model.predict(X).reshape((-1, 1))
    arr_centroids = kmeans_model.cluster_centers_

    """ store data to dataframe """
    dataframe = pd.DataFrame(X, columns=features_name)
    dataframe['clusters'] = arr_labels.flatten()

    # 2. Create an array of features and lables
    arr_features_and_labels = dataframe[features_name + ['clusters']].values

    # 3. Find the distance between the centroid and its respective vectors
    arr_distance_between_centroid_vectors = np.apply_along_axis(func1d=get_distance_centroid, arr=arr_features_and_labels,
                                                                axis=1, centroids=arr_centroids)
    dataframe['distance_centroid'] = arr_distance_between_centroid_vectors

    # 4. Ge the modified Z-score of each distance_centroid observation
    arr_distance_and_labels = dataframe[['distance_centroid', 'clusters']].values
    # -> Iterate over the labes and get the median and MAD
    arr_labels_iterations = np.sort(dataframe['clusters'].unique())
    list_medians = {}
    list_mads = {}
    for label in arr_labels_iterations:
        arr_distance_specific_cluster = dataframe.query(f"clusters == {label}")['distance_centroid'].values
        median_distance_specific_cluster = np.median(arr_distance_specific_cluster)
        mad_distance_specific_cluster = median_abs_deviation(arr_distance_specific_cluster)
        list_medians[label] = median_distance_specific_cluster
        list_mads[label] = mad_distance_specific_cluster

    # -> Calculate the Modified Z-score
    arr_z_mod = np.apply_along_axis(func1d=calculate_modifed_z_score, arr=arr_distance_and_labels, axis=1,
                                    medians=list_medians, mads=list_mads)
    dataframe['z_mod'] = arr_z_mod

    return dataframe

In [14]:
def detect_anomaly(value, threshold):
    if value > threshold:
        return -1 #? anomaly
    return 1 #? normal

In [15]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


def find_optimal_k_clusters_plot(X, minK, maxK, titlePlot, random_state=11):
    """
    This function displays a plot of the Within Cluster Sum Squares (WCSS) and the 
    Silhouette Score. The objective of this function is to visually identify the 
    optimal number of clusters

    * **`minK`** and **`maxK`** (_int_): are the minimun (inclusive) and maximum (non inclusive)
    amount of clusters to test.
    * **`random_state`** (_default = 11_): Is a random seed for reproducibility 
    """
    wcss = []
    list_models = []

    for i in range(minK, maxK):
        kmeans = KMeans(n_clusters=i, init='k-means++',
                        random_state=random_state)
        kmeans.fit(X)
        list_models.append(kmeans)
        wcss.append(kmeans.inertia_)

    sil_scores = [silhouette_score(X, model.labels_, random_state=random_state)
                  for model in list_models[1:]]

    # P L O T   R E S U L T S
    fig, ax1 = plt.subplots(figsize=(6, 4.5), dpi=100, facecolor='white')
    ax1.set_facecolor('white')
    fig.suptitle(titlePlot, size=13)
    ax1.plot(range(minK, maxK), wcss,
             linestyle='--', marker='P', color='teal', label='WCSS')
    ax1.set_xlabel('Number of clusters')
    ax1.set_ylabel('WCSS')
    ax1.tick_params(axis='y', colors='teal')
    ax1.yaxis.label.set_color('teal')

    ax2 = ax1.twinx()
    ax2.plot(range(minK + 1, maxK), sil_scores,
             linestyle=':', marker='o', color='maroon', label='Silhouette score')
    ax2.set_ylabel('Silhouette Score')
    ax2.tick_params(axis='y', colors='maroon')
    ax2.yaxis.label.set_color('maroon')


# Test and evaluate

In [16]:
from sklearn.metrics import classification_report, confusion_matrix, PrecisionRecallDisplay, f1_score, roc_auc_score, roc_curve, auc

def evaluate(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    print(classification_report(y_true, y_pred, digits=4))
    roc_auc = roc_auc_score(y_true, y_pred)
    print(roc_auc)

    # display = PrecisionRecallDisplay.from_predictions(y_true, y_pred)
    # _ = display.ax_.set_title("2-class Precision-Recall curve")
    return roc_auc

In [17]:
import pickle

def save_result(model, Xtr, Xt, model_name, selected_fts_names, roc_auc, expname='', config=None):
    ra = '{:.2f}'.format(round(roc_auc * 100, 2))
    if ra <= 70: #? not good enough to even bother
        return
    
    #? save model
    pickle.dump(model, open(f'{result_dir}/{af}5.model_{model_name}.__{ra}__.__{expname}__.{pf}.pkl','wb'))

    #? save transformed X as well
    np.save(f'{result_dir}/{af}5.data_{model_name}.__{ra}__.__{expname}__.tr.X.{pf}.npy', Xtr)
    np.save(f'{result_dir}/{af}5.data_{model_name}.__{ra}__.__{expname}__.t.X.{pf}.npy', Xt)
    # np.save(f'{result_dir}/u3-5.data_{model_name}.__{ra}__.v.X.{pf}.npy', x_val)

    #? save fts
    open(f'{result_dir}/{af}5.fts_{model_name}.__{ra}__.__{expname}__.{pf}.txt', 'w').write('\n'.join(selected_fts_names))
    #? save config
    if config is not None:
        open(f'{result_dir}/{af}5.config_{model_name}.__{ra}__.__{expname}__.{pf}.txt', 'w').write(config)

## Exp 02: Run on all features

The result increase, yet running time is much higher

```
CPU times: total: 2min 13s
Wall time: 39.8 s

[[751564  10016]
 [  1123   1839]]
              precision    recall  f1-score   support

           0     0.9985    0.9868    0.9926    761580
           1     0.1551    0.6209    0.2482      2962

    accuracy                         0.9854    764542
   macro avg     0.5768    0.8039    0.6204    764542
weighted avg     0.9952    0.9854    0.9898    764542

0.8038563375096434
```

In [18]:
""" select features """
selected_fts_names__02 = fts_names[:]
', '.join(selected_fts_names__02)

'Dur, sTos, dTos, Sport, Dport, TotPkts, TotBytes, SrcBytes, PktsPerSec, BytesPerSec, SrcBytesPerSec, BytesPerPkt, DstBytes, DstBytesPerSec, SportHex, DportHex, sTos_-1, sTos_0, sTos_1, sTos_2, sTos_3, dTos_-1, dTos_0, dTos_1, dTos_2, dTos_3, State_CON, State_alltcp, State_INT, State_S_, State_URP, State_ECO, State_RED, State_REQ, State_ECR, State_URH, State_TXD, State_URFIL, State_R_, State_URN, State_RSP, State_URHPRO, State_A_, State_other, Flag_nan, Flag_S, Flag_A, Flag_P, Flag_R, Flag_F, Proto_udp, Proto_tcp, Proto_icmp, Proto_rtp, Proto_rtcp, Proto_igmp, Proto_arp, Proto_other, Service_80, Service_443, Service_21, Service_22, Service_25, Service_6667, Service_other'

In [19]:
X_train__02 = Xtrain[:,:]
X_test__02 = Xtest[:,:]
# X_val__02 = Xval[:,:]
print(X_train__02.shape)

(1529165, 65)


In [20]:
# """ Test from 1 to 12 clusters """
# find_optimal_k_clusters_plot(X_train__02, minK=1, maxK=4, titlePlot='Find optimal k')

In [22]:
""" train """
from sklearn.cluster import KMeans

n_clusters = 6
model__02 = KMeans(n_clusters=n_clusters, init='random', n_init=10, max_iter=500000, random_state=11, verbose=0)
model__02.fit(X_train__02)

%time model__02.fit(X_train__02)

CPU times: total: 1min 40s
Wall time: 7.9 s


KMeans(init='random', max_iter=500000, n_clusters=6, random_state=11)

In [51]:
%%time
""" predict """
df_out__02 = characterization_k_clusters(model__02, X_test__02, features_name=selected_fts_names__02)

CPU times: total: 11.1 s
Wall time: 10.3 s


In [52]:
print(df_out__02['z_mod'].describe())

count    764542.000000
mean          0.504369
std           5.202642
min          -3.640574
25%          -0.871671
50%           0.000000
75%           1.074429
max         440.155625
Name: z_mod, dtype: float64


In [53]:
""" detect outliers using threshold on z_mod score """
thresh = 20
df_out__02['Label_Pred'] = df_out__02['z_mod'].apply(detect_anomaly, args=(thresh,))
y_pred__02 = df_out__02['Label_Pred'].values

In [54]:
""" evaluate """
#? relabel to use classification metrics
y_pred__02[y_pred__02 == 1] = 0
y_pred__02[y_pred__02 == -1] = 1

roc_auc__02 = evaluate(y_test, y_pred__02)

[[759422   2158]
 [  2962      0]]
              precision    recall  f1-score   support

           0     0.9961    0.9972    0.9966    761580
           1     0.0000    0.0000    0.0000      2962

    accuracy                         0.9933    764542
   macro avg     0.4981    0.4986    0.4983    764542
weighted avg     0.9923    0.9933    0.9928    764542

0.49858320859266264


### Save model ?

In [20]:
save_result(model__02, X_train__02, X_test__02, 'iforest', selected_fts_names__02, roc_auc__02, 'exp02__fts-all')

## Exp 04: Run on chosen set of features

The overall ROC_AUC end execution time does not change much from experiment 02 or 03.  
Recall for minority class increases.  

```
CPU times: total: 7h 17min 18s
Wall time: 27min 47s

[[744296  17284]
 [  1711   1251]]
              precision    recall  f1-score   support

           0     0.9977    0.9773    0.9874    761580
           1     0.0675    0.4223    0.1164      2962

    accuracy                         0.9752    764542
   macro avg     0.5326    0.6998    0.5519    764542
weighted avg     0.9941    0.9752    0.9840    764542

0.6998274199809809
```

In [42]:
""" select features """
selected_fts_names__04 = ['sTos', 'Sport', 'Dport', 'TotPkts', 'TotBytes', 'SrcBytes', 'PktsPerSec', 'BytesPerSec', 'SrcBytesPerSec', 'BytesPerPkt', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other']
idx__selected_fts_names__04 = [fts_names.index(s) for s in selected_fts_names__04]
len(idx__selected_fts_names__04)

49

In [43]:
X_train__04 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)
X_test__04 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)

print(X_train__04.shape)

(1529165, 49)


In [44]:
""" train """
from sklearn.cluster import KMeans

n_clusters = 6
model__04 = KMeans(n_clusters=n_clusters, init='random', n_init=10, max_iter=500000, random_state=11, verbose=0)
model__04.fit(X_train__04)

%time model__04.fit(X_train__04)

CPU times: total: 1min 1s
Wall time: 5.14 s


KMeans(init='random', max_iter=500000, n_clusters=6, random_state=11)

In [45]:
%%time
""" predict """
df_out__04 = characterization_k_clusters(model__04, X_test__04, features_name=selected_fts_names__04)

CPU times: total: 10.2 s
Wall time: 9.7 s


In [50]:
print(df_out__04['z_mod'].describe())

count    764542.000000
mean          0.257282
std           2.320004
min          -4.776297
25%          -0.985876
50%           0.000000
75%           1.028351
max          26.539831
Name: z_mod, dtype: float64

In [46]:
""" detect outliers using threshold on z_mod score """
thresh = 2.5
df_out__04['Label_Pred'] = df_out__04['z_mod'].apply(detect_anomaly, args=(thresh,))
y_pred__04 = df_out__04['Label_Pred'].values

In [47]:
""" evaluate """
#? relabel to use classification metrics
y_pred__04[y_pred__04 == 1] = 0
y_pred__04[y_pred__04 == -1] = 1

roc_auc__04 = evaluate(y_test, y_pred__04)

[[734784  26796]
 [  1009   1953]]
              precision    recall  f1-score   support

           0     0.9986    0.9648    0.9814    761580
           1     0.0679    0.6594    0.1232      2962

    accuracy                         0.9636    764542
   macro avg     0.5333    0.8121    0.5523    764542
weighted avg     0.9950    0.9636    0.9781    764542

0.8120835209164556


### Save model ?

In [48]:
save_result(model__04, X_train__04, X_test__04, 'kmeans', selected_fts_names__04, roc_auc__04, 'exp04__play', config=f"n_clusters={n_clusters}, init='random', n_init=10, max_iter=500000\nthresh={thresh}")

## Exp 06: Run on features chosen from MI

**Chosen features from MI: `State_S_` ,` dTos` ,` Proto_tcp` ,` Proto_udp` ,` Service_80` ,` Service_other` ,` State_CON` ,` Service_25` ,` Sport` ,` Flag_S` ,` Flag_R` ,` BytesPerPkt` ,` Dur` ,` Service_6667` ,` BytesPerSec` ,` State_alltcp` ,` DstBytesPerSec` ,` Flag_A` ,` PktsPerSec` ,` SrcBytesPerSec`**

```
CPU times: total: 2min 42s
Wall time: 37.3 s

[[752538   9042]
 [  1486   1476]]
              precision    recall  f1-score   support

           0     0.9980    0.9881    0.9931    761580
           1     0.1403    0.4983    0.2190      2962

    accuracy                         0.9862    764542
   macro avg     0.5692    0.7432    0.6060    764542
weighted avg     0.9947    0.9862    0.9901    764542

0.7432196328259532
```

In [34]:
""" select features """
# selected_fts_names__06 = fts_names[29:30] + fts_names[2:3] + fts_names[51:52] + fts_names[50:51] + fts_names[58:59] + fts_names[64:65] + fts_names[26:27] + fts_names[62:63] + fts_names[3:4] + fts_names[45:46] + fts_names[48:49] + fts_names[26:65]
selected_fts_names__06 = ['State_S_','dTos','Proto_tcp','Proto_udp','Service_80','Service_other','State_CON','Service_25','Sport','Flag_S','Flag_R','BytesPerPkt','Dur','Service_6667','BytesPerSec','State_alltcp','DstBytesPerSec','Flag_A','PktsPerSec','SrcBytesPerSec']
idx__selected_fts_names__06 = [fts_names.index(s) for s in selected_fts_names__06]
len(idx__selected_fts_names__06)
# ', '.join(selected_fts_names__05)

20

In [35]:
X_train__06 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)
X_test__06 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)

print(X_train__06.shape)

(1529165, 20)


In [36]:
""" train """
from sklearn.ensemble import IsolationForest

n_estimators = 35
contamination = 0.01
model__06 = IsolationForest(n_estimators=n_estimators, max_samples=X_train__06.shape[0], contamination=contamination, n_jobs=10, verbose=1)

%time model__06.fit(X_train__05)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:   13.0s remaining:   52.4s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   16.1s finished


CPU times: total: 2min 42s
Wall time: 37.3 s


IsolationForest(contamination=0.01, max_samples=1529165, n_estimators=35,
                n_jobs=10, verbose=1)

In [37]:
%%time
""" predict """
y_pred__06 = model__06.predict(X_test__06)

CPU times: total: 10.2 s
Wall time: 10.3 s


In [38]:
""" evaluate """
#? relabel to use classification metrics
y_pred__06[y_pred__06 == 1] = 0
y_pred__06[y_pred__06 == -1] = 1

roc_auc__06 = evaluate(y_test, y_pred__06)

[[752538   9042]
 [  1486   1476]]
              precision    recall  f1-score   support

           0     0.9980    0.9881    0.9931    761580
           1     0.1403    0.4983    0.2190      2962

    accuracy                         0.9862    764542
   macro avg     0.5692    0.7432    0.6060    764542
weighted avg     0.9947    0.9862    0.9901    764542

0.7432196328259532


### Save model ?

In [39]:
save_result(model__06, X_train__06, X_test__06, 'iforest', selected_fts_names__06, roc_auc__06, 'exp06__mi')

## Checking score

The best set of features is of experiment 02.  
Use this experiment result to further analyze the scores of outliers

Check `vv2-6.after_predict`